In [305]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [307]:
import csv
import torch
fileName = 'data/loan_data.csv'

with open(fileName) as csvFile:
    reader = csv.reader(csvFile)
    variables = next(reader)
    columnInfo = [[] for _ in range(len(variables))]
    data = torch.tensor([])
    labels = torch.tensor([])
    for row in reader:
        temp = torch.zeros(1, len(variables) - 1)
        for columnIndex, column in enumerate(row[:-1]):
            try:
                temp[0][columnIndex] = float(column)
            except ValueError:
                if column not in columnInfo[columnIndex]:
                    columnInfo[columnIndex].append(column)
                temp[0][columnIndex] = columnInfo[columnIndex].index(column)
        data = torch.cat((data, temp))
        labels = torch.cat((labels, torch.tensor([int(row[-1])])))
labels = labels
data.shape, labels.shape

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df = df.set_axis(variables[:-1], axis=1)
df

In [ ]:
pct = .8
XSplit = int(pct*len(data))
ySplit = int(pct * len(labels))
XTrain, XTest = data[:XSplit].to(device), data[XSplit:].to(device)
yTrain, yTest = labels[:ySplit].to(device), labels[ySplit:].to(device)
XTrain.shape, XTest.shape, yTrain.shape, yTest.shape

In [ ]:
XTrain[0:5], yTrain[0:5]

In [ ]:
from torch import nn
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(13, 20)
        self.layer2 = nn.Linear(20, 10)
        self.layer3 = nn.Linear(10, 1)
    def forward(self, x):
        return self.layer3(self.layer2(self.layer1(x)))
model = Model0().to(device)
lossFun = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters())
epochs = 11
for epoch in range(epochs):
    model.train()
    yLogits = model(XTrain).squeeze()
    yPred = torch.round(torch.sigmoid(yLogits))
    trainLoss = lossFun(yLogits, yTrain)
    # acc = len(torch.unique(yLogits[testLogits == yTest])) / len(yTrain)
    optimizer.zero_grad()
    trainLoss.backward()
    optimizer.step()
    model.eval()
    with torch.inference_mode():
        testLogits = model(XTest).squeeze()
        testPred = torch.round(torch.sigmoid(testLogits))
        testLoss = lossFun(testLogits, yTest)
        # testAcc = len(torch.unique(testPred[testPred == yTest])) / len(yTest)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02} | Train loss: {trainLoss:.4f} | Test loss: {testLoss:.4f}')

In [ ]:
model.eval()
with torch.inference_mode():
    print(XTest[0], yTest[0], model(XTest[0]))

In [ ]:
yLogits